# **Design of a Machine Learning-Based Anomaly Detection for Medical Prescription Data**



#Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

#Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019.zip

Archive:  /content/drive/MyDrive/Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019.zip
  inflating: Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019.csv  


#Dask

In [ ]:
!python -m pip install dask 
!python -m pip install dask[dataframe] --upgrade 

In [ ]:
import dask.dataframe as dd

#used Dask to first look into the data
df3 = dd.read_csv('/content/*.csv')
print(df3.columns)
df3

['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_State_FIPS',
       'Prscrbr_Type', 'Prscrbr_Type_Src', 'Brnd_Name', 'Gnrc_Name',
       'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst',
       'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills',
       'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag',
       'GE65_Tot_Benes']

#Load Medicare Part D Dataset


In [ ]:
#columns used
columns = ['Prscrbr_NPI','Prscrbr_Type','Brnd_Name','Tot_Clms',
           'Tot_30day_Fills','Tot_Day_Suply','Tot_Drug_Cst','Tot_Benes']

#change datatype
dtypes = {'Prscrbr_Type':'category',
          'Brnd_Name':'category'}

#rename columns
columns_rename = {'Prscrbr_NPI':'NPI', 'Prscrbr_Type':'provider_type',
                   'Brnd_Name':'brand_name', 'Tot_Clms':'num_claims',
                   'Tot_30day_Fills':'num_30day_refills','Tot_Day_Suply':'agg_daysupply',
                   'Tot_Drug_Cst':'cost','Tot_Benes':'num_beneficiaries'}

#load data
df = pd.read_csv("/content/Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019.csv", usecols = columns, dtype=dtypes)

#rename columns
df.rename(columns=columns_rename, inplace=True)

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25401870 entries, 0 to 25401869
Data columns (total 8 columns):
 #   Column             Dtype   
---  ------             -----   
 0   NPI                int64   
 1   provider_type      category
 2   brand_name         category
 3   num_claims         int64   
 4   num_30day_refills  float64 
 5   agg_daysupply      int64   
 6   cost               float64 
 7   num_beneficiaries  float64 
dtypes: category(2), float64(3), int64(3)
memory usage: 1.2 GB
None


,NPI,provider_type,brand_name,num_claims,num_30day_refills,agg_daysupply,cost,num_beneficiaries
0,1003000126,Internal Medicine,Amiodarone Hcl,13,13.0,360,150.76,NaN
1,1003000126,Internal Medicine,Amlodipine Besylate,22,26.0,780,92.81,12.0
2,1003000126,Internal Medicine,Atorvastatin Calcium,41,50.0,1482,373.60,22.0
3,1003000126,Internal Medicine,Azithromycin,15,15.0,77,132.57,15.0
4,1003000126,Internal Medicine,Carvedilol,21,27.0,787,118.53,11.0


In [ ]:
#drop NaN and reset index
df.dropna(how='any', inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10706142 entries, 0 to 10706141
Data columns (total 8 columns):
 #   Column             Dtype   
---  ------             -----   
 0   NPI                int64   
 1   provider_type      category
 2   brand_name         category
 3   num_claims         int64   
 4   num_30day_refills  float64 
 5   agg_daysupply      int64   
 6   cost               float64 
 7   num_beneficiaries  float64 
dtypes: category(2), float64(3), int64(3)
memory usage: 531.0 MB


# Aggregate Medicare Part D

In [ ]:
# all unique npi
df["NPI"].nunique()

827474

In [ ]:
# Using GroupBy multiple column
df = df.groupby(["NPI", "provider_type","brand_name"],observed=True).agg({"num_claims": "sum", "num_30day_refills": "sum", "agg_daysupply": "sum","cost": "sum","num_beneficiaries": "sum"}).reset_index()

In [ ]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10699504 entries, 0 to 10699503
Data columns (total 8 columns):
 #   Column             Dtype   
---  ------             -----   
 0   NPI                int64   
 1   provider_type      category
 2   brand_name         category
 3   num_claims         int64   
 4   num_30day_refills  float64 
 5   agg_daysupply      int64   
 6   cost               float64 
 7   num_beneficiaries  float64 
dtypes: category(2), float64(3), int64(3)
memory usage: 530.7 MB
None


,NPI,provider_type,brand_name,num_claims,num_30day_refills,agg_daysupply,cost,num_beneficiaries
0,1003000126,Internal Medicine,Amlodipine Besylate,22,26.0,780,92.81,12.0
1,1003000126,Internal Medicine,Atorvastatin Calcium,41,50.0,1482,373.60,22.0
2,1003000126,Internal Medicine,Azithromycin,15,15.0,77,132.57,15.0
3,1003000126,Internal Medicine,Carvedilol,21,27.0,787,118.53,11.0
4,1003000126,Internal Medicine,Cefdinir,13,13.0,81,203.03,13.0


#Load List of Excluded Individuals and Entities

In [ ]:
df_leie = pd.read_csv('/content/drive/MyDrive/LEIE_Tiz.csv', usecols = ['NPI', 'EXCLTYPE'])
print(df_leie.info())
df_leie.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75164 entries, 0 to 75163
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NPI       75164 non-null  int64 
 1   EXCLTYPE  75164 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB
None


,NPI,EXCLTYPE
0,0,1128a1
1,0,1128a1
2,0,1128b7
3,1922348218,1128a1
4,0,1128b5


In [ ]:
#rename exclusion type column and drop all 0 values in the NPI column
df_leie.rename(columns={'EXCLTYPE': 'fraud'}, inplace=True)
df_leie.query('NPI !=0', inplace=True)
df_leie.reset_index(drop=True, inplace=True)
df_leie['fraud'] = 1
df_leie.head()

,NPI,fraud
0,1922348218,1
1,1942476080,1
2,1275600959,1
3,1891731758,1
4,1851631543,1


#Merge Part D and LEIE

In [ ]:
df_merge = pd.merge(df, df_leie, on='NPI', how='left')

In [ ]:
# for whole dataframe the nan values are replaced by zeros
df_merge = df_merge.replace(np.nan, 0)

In [ ]:
df_merge['fraud']=df_merge.fraud.astype('int64') #changes float type to int type

In [ ]:
df_merge[df_merge['fraud']==1].count()
df_merge[df_merge['fraud']==1]
df_merge["fraud"].value_counts()

0    10696346
1        3158
Name: fraud, dtype: int64



There are 3162 drug related fraud in the dataset before aggregation and 3158 after aggregation


In [ ]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10699504 entries, 0 to 10699503
Data columns (total 9 columns):
 #   Column             Dtype   
---  ------             -----   
 0   NPI                int64   
 1   provider_type      category
 2   brand_name         category
 3   num_claims         int64   
 4   num_30day_refills  float64 
 5   agg_daysupply      int64   
 6   cost               float64 
 7   num_beneficiaries  float64 
 8   fraud              int64   
dtypes: category(2), float64(3), int64(4)
memory usage: 694.0 MB


In [ ]:
df_merge["provider_type"].nunique()

157

In [ ]:
df_merge["NPI"].nunique()

827474

#Save Pickel


In [ ]:
joblib.dump(df_merge, open("final_merge_provider.pkl",'wb'))
#rfc_loaded = load("rfc_fitting.pkl") #to open